In [ ]:
from typing import Dict, List, Optional
from util import llm_call, extract_xml

def parse_tasks(tasks_xml: str) -> List[Dict]:
    """Parse XML tasks into a list of task dictionaries."""
    tasks = []
    current_task = {}
    
    for line in tasks_xml.split('\n'):
        line = line.strip()
        if not line:
            continue
            
        if line.startswith("<task>"):
            current_task = {}
        elif line.startswith("<type>"):
            current_task["type"] = line[6:-7].strip()
        elif line.startswith("<description>"):
            current_task["description"] = line[12:-13].strip()
        elif line.startswith("</task>"):
            if "description" in current_task:
                if "type" not in current_task:
                    current_task["type"] = "default"
                tasks.append(current_task)
    
    return tasks

class FlexibleOrchestrator:
    """Break down tasks and run them in parallel using worker LLMs."""
    
    def __init__(
        self,
        orchestrator_prompt: str,
        worker_prompt: str,
    ):
        """Initialize with prompt templates."""
        self.orchestrator_prompt = orchestrator_prompt
        self.worker_prompt = worker_prompt

    def _format_prompt(self, template: str, **kwargs) -> str:
        """Format a prompt template with variables."""
        try:
            return template.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Missing required prompt variable: {e}")

    def process(self, task: str, context: Optional[Dict] = None) -> Dict:
        """Process task by breaking it down and running subtasks in parallel."""
        context = context or {}
        
        # Step 1: Get orchestrator response
        orchestrator_input = self._format_prompt(
            self.orchestrator_prompt,
            task=task,
            **context
        )
        orchestrator_response = llm_call(orchestrator_input)
        
        # Parse orchestrator response
        analysis = extract_xml(orchestrator_response, "analysis")
        tasks_xml = extract_xml(orchestrator_response, "tasks")
        tasks = parse_tasks(tasks_xml)
        
        print("\n=== ORCHESTRATOR OUTPUT ===")
        print(f"\nANALYSIS:\n{analysis}")
        print(f"\nTASKS:\n{tasks}")
        
        # Step 2: Process each task
        worker_results = []
        for task_info in tasks:
            worker_input = self._format_prompt(
                self.worker_prompt,
                original_task=task,
                task_type=task_info['type'],
                task_description=task_info['description'],
                **context
            )
            
            worker_response = llm_call(worker_input)
            result = extract_xml(worker_response, "response")
            
            worker_results.append({
                "type": task_info["type"],
                "description": task_info["description"],
                "result": result
            })
            
            print(f"\n=== WORKER RESULT ({task_info['type']}) ===\n{result}\n")
        
        return {
            "analysis": analysis,
            "worker_results": worker_results,
        }

=== ORCHESTRATOR OUTPUT ===

ANALYSIS:

The task requires writing a product description for an eco-friendly water bottle, which involves highlighting its sustainability, functionality, and appeal to the target audience. Different approaches can serve different purposes:  

1. **Formal Approach**: This would focus on technical details, materials, and specifications, appealing to logical buyers who prioritize facts and features. It’s useful for B2B audiences, eco-conscious consumers who research thoroughly, or platforms where specifications are key (e.g., e-commerce product pages).  

2. **Conversational Approach**: This would emphasize storytelling, emotional connection, and relatability, targeting everyday consumers who care about sustainability but also want a product that fits their lifestyle. It’s ideal for social media, blog posts, or direct-to-consumer marketing.  

A third possible approach could be **lifestyle-centric**, blending sustainability with aspirational messaging (e.g., "Your perfect gym companion"), but the first two cover the core needs of clarity and engagement.  


TASKS:
[{'type': 'formal', 'description': '>Write a precise, technical version that emphasizes specifications like material composition (e.g., "Made from 100% recycled stainless steel"), capacity, weight, leak-proof features, and certifications (e.g., BPA-free, carbon-neutral). Highlight durability and environmental impact with data.<'}, {'type': 'conversational', 'description': '>Write an engaging, friendly version that connects with readers through relatable benefits (e.g., "Stay hydrated while saving the planet—one sip at a time"). Use casual language, anecdotes, or humor to showcase the bottle\'s everyday use and eco-friendly mission.<'}]

=== WORKER RESULT (formal) ===

**EcoVista Premium Stainless Steel Water Bottle**  

The **EcoVista Premium Stainless Steel Water Bottle** is a meticulously engineered hydration solution designed for sustainability and performance. Crafted from **100% recycled 18/8 stainless steel**, this bottle combines durability with environmental responsibility, reducing landfill waste and conserving resources.  

**Key Specifications:**  
- **Capacity:** 750ml (25.4 oz), optimized for daily hydration needs.  
- **Weight:** 320g (11.3 oz), balancing portability with robust construction.  
- **Material Composition:** High-grade 18/8 stainless steel, ensuring corrosion resistance and longevity.  
- **Leak-Proof Design:** Precision-engineered twist-off cap with silicone gasket for a secure, spill-proof seal.  
- **Temperature Retention:** Maintains liquid temperature for up to **12 hours (hot)** or **24 hours (cold)**.  

**Environmental Credentials:**  
- **BPA-Free & Non-Toxic:** Certified to meet FDA and EU food-contact safety standards.  
- **Carbon-Neutral Production:** Manufactured in a facility powered by renewable energy, with carbon offsets applied to neutralize emissions.  
- **Recyclability:** 100% recyclable at end of life, supporting a circular economy.  

**Durability:** Tested to withstand impacts and drops up to **1.5 meters**, backed by a **10-year warranty** against material defects.  

Ideal for eco-conscious consumers, the EcoVista bottle aligns with rigorous sustainability goals while delivering unmatched functionality. Choose a hydration companion that prioritizes the planet without compromising quality.  



=== WORKER RESULT (conversational) ===

**Meet Your New Hydration Hero: The EcoSip Bottle**  

Tired of lugging around boring, guilt-inducing plastic bottles? Say hello to the **EcoSip**—your stylish sidekick for staying hydrated *and* saving the planet, one sip at a time.  

Made from durable, lightweight stainless steel (and 100% BPA-free, because who wants mystery chemicals in their water?), this bottle keeps your drinks icy cold for 24 hours or steaming hot for 12. Morning coffee, post-workout hydration, or even that 3 p.m. iced tea slump—EcoSip’s got your back.  

But here’s the best part: every time you refill, you’re dodging another plastic bottle from clogging up landfills or chilling in the ocean (sorry, plastic straws, you’re *not* invited). Plus, with its sleek design and leak-proof lid, this bottle is as ready for your desk as it is for your weekend adventures.  

So go ahead, hydrate like it’s your job—because with EcoSip, every sip is a high-five to Mother Earth. 🌎💧  

*P.S. The planet (and your future self) will thank you.*  